**Построение модели классификации**
1. Обзор обучающего датасета
2. Обработка выбросов
3. Обработка пропусков
4. Анализ данных
5. Отбор признаков
6. Балансировка классов
7. Подбор моделей, получение бейзлана
8. Выбор наилучшей модели, настройка гиперпараметров
9. Проверка качества, борьба с переобучением
10. Интерпретация результатов

**Прогнозирование на тестовом датасете**
1. Выполнить для тестового датасета те же этапы обработки и постронияния признаков
2. Спрогнозировать целевую переменную, используя модель, построенную на обучающем датасете
3. Прогнозы должны быть для всех примеров из тестового датасета (для всех строк)
4. Соблюдать исходный порядок примеров из тестового датасета

**Описание датасета**

* **Home Ownership** - домовладение
* **Annual Income** - годовой доход
* **Years in current job** - количество лет на текущем месте работы
* **Tax Liens** - налоговые обременения
* **Number of Open Accounts** - количество открытых счетов
* **Years of Credit History** - количество лет кредитной истории
* **Maximum Open Credit** - наибольший открытый кредит
* **Number of Credit Problems** - количество проблем с кредитом
* **Months since last delinquent** - количество месяцев с последней просрочки платежа
* **Bankruptcies** - банкротства
* **Purpose** - цель кредита
* **Term** - срок кредита
* **Current Loan Amount** - текущая сумма кредита
* **Current Credit Balance** - текущий кредитный баланс
* **Monthly Debt** - ежемесячный долг
* **Credit Default** - факт невыполнения кредитных обязательств (0 - погашен вовремя, 1 - просрочка)

**Пути к директориям и файлам**

In [270]:
TRAIN_DATASET_PATH = 'course_project_train.csv'
TEST_DATASET_PATH = 'course_project_test.csv'

**Загрузка данных**

In [271]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression




In [272]:
train_df = pd.read_csv(TRAIN_DATASET_PATH)
test_df = pd.read_csv(TEST_DATASET_PATH)
train_df.head()

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
0,Own Home,482087.0,NaN,0.0,11.0,26.3,685960.0,1.0,NaN,1.0,debt consolidation,Short Term,99999999.0,47386.0,7914.0,749.0,0
1,Own Home,1025487.0,10+ years,0.0,15.0,15.3,1181730.0,0.0,NaN,0.0,debt consolidation,Long Term,264968.0,394972.0,18373.0,737.0,1
2,Home Mortgage,751412.0,8 years,0.0,11.0,35.0,1182434.0,0.0,NaN,0.0,debt consolidation,Short Term,99999999.0,308389.0,13651.0,742.0,0
3,Own Home,805068.0,6 years,0.0,8.0,22.5,147400.0,1.0,NaN,1.0,debt consolidation,Short Term,121396.0,95855.0,11338.0,694.0,0
4,Rent,776264.0,8 years,0.0,13.0,13.6,385836.0,1.0,NaN,0.0,debt consolidation,Short Term,125840.0,93309.0,7180.0,719.0,0


In [273]:
# Удаление неинформативных столбцов
df_train = train_df.drop(columns=['Months since last delinquent'])

In [274]:
# Разделение данных на признаки и целевую переменную
X = train_df.drop('Credit Default', axis=1)
y = train_df['Credit Default']

In [275]:

# Разделение признаков на числовые и категориальные
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

Для заполнения пропущенных значений, масштабирования числовых значений и кодирования категориальных признаков будем использовать **Pipeline**

Основные преимущества Pipeline:

- На входе можно подать абсолютно «сырые» данные, которые будут обработаны внутри pipeline, в результате чего на выходе будет получен необходимый результат (predict).

- Обработка данных происходит сразу как для обучающей выборки, так и для тестовой, что избавляет от дублирования кода и уменьшает вероятность совершения ошибок в коде.

In [276]:
# Создание трансформеров для числовых и категориальных данных
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # Заполнение пропущенных значений медианным
    ('scaler', StandardScaler())]) # Масштабирование данных

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # Заполнение пропущенных значений наиболее частым значением
    ('onehot', OneHotEncoder(handle_unknown='ignore'))]) # Игнорирование новых (неизвестных) категорий

In [277]:
# Объединение трансформеров в ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [278]:
# Применение предварительной обработки к данным
X = preprocessor.fit_transform(X)

In [279]:
# Оценим сбалансированность классов
train_df['Credit Default'].value_counts()

Credit Default
0    5387
1    2113
Name: count, dtype: int64

In [280]:
# Проведем балансировку классов путем увеличения меньшего класса
class_0 = train_df[train_df['Credit Default'] == 0]
class_1 = train_df[train_df['Credit Default'] == 1]

In [281]:
# Увеличим количество примеров меньшего класса до размера большего класса с помощью метода sample
class_1_upsampled = class_1.sample(len(class_0), replace=True, random_state=42)
train_df_balanced = pd.concat([class_0, class_1_upsampled])

In [282]:
# Обновление X и y после балансировки
X_balanced = train_df_balanced.drop('Credit Default', axis=1)
y_balanced = train_df_balanced['Credit Default']

In [283]:
# Применение предварительной обработки к сбалансированным данным
X_balanced = preprocessor.transform(X_balanced)

In [284]:

# Разделение на тренировочную и тестовую выборки
X_train, X_val, y_train, y_val = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

In [285]:
# Модель с весами классов
model = LogisticRegression(class_weight='balanced', solver='lbfgs', random_state=42, max_iter=1000) # установка весов классов в модели (class_weight='balanced') может дополнительно помочь корректировать оставшийся дисбаланс.
param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1')
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

In [286]:
# Оценка модели на валидационном наборе данных
y_train_pred = best_model.predict(X_train)
print("Отчет c метриками для классов из тренировочного датасета")
print(classification_report(y_train, y_train_pred))
print("Матрица набора тренировочных данных")
print(confusion_matrix(y_train, y_train_pred))

Отчет c метриками для классов из тренировочного датасета
              precision    recall  f1-score   support

           0       0.68      0.72      0.70      4320
           1       0.70      0.66      0.68      4299

    accuracy                           0.69      8619
   macro avg       0.69      0.69      0.69      8619
weighted avg       0.69      0.69      0.69      8619

Матрица набора тренировочных данных
[[3099 1221]
 [1472 2827]]


In [287]:
# Оценка модели на валидационном наборе данных
y_val_pred = best_model.predict(X_val)
print("Отчет c метриками для классов из тренировочного датасета ")
print(classification_report(y_val, y_val_pred))
print("Матрица набора валидационных данных")
print(confusion_matrix(y_val, y_val_pred))

Отчет c метриками для классов из тренировочного датасета 
              precision    recall  f1-score   support

           0       0.66      0.70      0.68      1067
           1       0.69      0.65      0.67      1088

    accuracy                           0.68      2155
   macro avg       0.68      0.68      0.68      2155
weighted avg       0.68      0.68      0.68      2155

Матрица набора валидационных данных
[[750 317]
 [380 708]]


Так как метрика F1-score показала на валидационной выборке хороший результат для главного класса (F1>0.5) и показатели не упали по сравнению с тренировочными(переобучение миновало), можно применить полученную модель прогнозирования для тестового набора данных

In [288]:
# Прогнозирование на тестовом наборе данных
X_test = preprocessor.transform(test_df)
y_test_pred = best_model.predict(X_test)

In [289]:
test_df['Credit Default Predict'] = y_test_pred

In [290]:
test_df.to_csv("TrifonovP_predictions.csv")